In [1]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [2]:
# !pip install import-ipynb

In [3]:
import import_ipynb

In [4]:
# %cd /content/drive/MyDrive/A62_projet_synthese_feature/feature-colab-from-git/notebook

In [5]:
%run iteration_01_0_imports.py

In [6]:
%run iteration_01_0_parameters.py

In [7]:
import iteration_01_0_utils as utils

importing Jupyter notebook from iteration_01_0_utils.ipynb


In [8]:
%whos

Variable           Type      Data/Info
--------------------------------------
HEALTHY            str       H
IMG_SIZE           int       64
Image              module    <module 'PIL.Image' from <...>packages\\PIL\\Image.py'>
NBR_SAMPLE         int       20000
PATH_DATA          str       ../data
PATH_DATA_EDA      str       ../data/eda
PATH_DATA_LIST     str       ../data/list
PATH_DATA_SAMPLE   str       ../data/sample
PATH_MODEL         str       ../model
PATH_ROOT          str       ..
PATH_SOURCE_IMG    str       ../cell_images
SICK               str       S
cv2                module    <module 'cv2' from 'C:\\U<...>kages\\cv2\\__init__.py'>
datetime           type      <class 'datetime.datetime'>
glob               module    <module 'glob' from 'C:\\<...>anaconda3\\lib\\glob.py'>
gzip               module    <module 'gzip' from 'C:\\<...>anaconda3\\lib\\gzip.py'>
import_ipynb       module    <module 'import_ipynb' fr<...>ckages\\import_ipynb.py'>
np                 module    <mod

In [9]:
df_path_img_sick = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_sick_img.csv")
df_path_img_healthy = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_healthy_img.csv")
df_path_img = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_img.csv")

# Prétraitement

## positif_global_std

In [10]:
IMG_IN_COLOR = 1
def image_resize(path_img: str) -> np.array:
    img_arr = cv2.imread(path_img, flags = IMG_IN_COLOR)
    
    h, w, _ = img_arr.shape
    ratio = IMG_SIZE / max(h,w) 
    img_arr_resize = cv2.resize(img_arr, dsize = (int(np.ceil(w*ratio)), int(np.ceil(h*ratio)) ))
    return img_arr_resize

In [11]:
def positive_global_std(img_arr: np.array) -> np.array:
    
    mean = img_arr.mean()
    std  = img_arr.std()
    img_arr_std = (img_arr - mean) / std
    
    # clip les valeurs
    img_arr_std = np.clip(img_arr_std, -1.0, 1.0)
    
    # shift to 0--1
    img_arr_std = (img_arr_std + 1.0) / 2.0
    
    return img_arr_std

* Voir avec le prof la pertinence de clip les valeurs extremes -1 et 1

In [12]:
NBR_SAMPLE

20000

In [13]:
df_path_img_healthy_sample = df_path_img_healthy.sample(n = int(NBR_SAMPLE/2))
df_path_img_sick_sample    = df_path_img_sick.sample(n = int(NBR_SAMPLE/2))

In [14]:
PATH_DATA_SAMPLE

'../data/sample'

In [15]:
DATE_TIME = "2021-11-18_20-39-00"

def create_sample(df_path_img:str, img_type:str, path_sample:str, date_time:str, clean_dir:bool):
    path_sample_time = path_sample+"/"+date_time

    os.makedirs(path_sample_time, exist_ok=True)
    if clean_dir:
        file_list = glob.glob(path_sample_time+"/"+"*.*")
        for f in file_list:
            os.remove(f)
    
    for path_img , _ in zip(df_path_img.path_img, tqdm(range(len(df_path_img.path_img)))):
        
        _, img_name, _ = utils.split_file_info(path_img)
        
        
        img_arr_std = positive_global_std(image_resize(path_img))
        # padding
        img_arr_padded = np.zeros((IMG_SIZE, IMG_SIZE, 3))
        img_arr_padded[:img_arr_std.shape[0],:img_arr_std.shape[1],:] = img_arr_std
        
        utils.pickle_save(img_arr_padded, path_sample_time + "/" + str(img_type).upper() + "_" + img_name + ".pklz")
        

In [16]:
create_sample(df_path_img_healthy_sample, HEALTHY, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=True)

100%|█████████▉| 9999/10000 [05:06<00:00, 32.62it/s]


In [17]:
create_sample(df_path_img_sick_sample, SICK, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=False)

100%|█████████▉| 9999/10000 [05:08<00:00, 32.39it/s]


In [18]:
df_path_img_sample = pd.concat([df_path_img_healthy_sample, df_path_img_sick_sample], ignore_index=True)
df_path_img_not_in_sample = pd.DataFrame([i for i in df_path_img.path_img.values if not i in df_path_img_sample.path_img.values], 
                 columns=df_path_img_sample.columns)

In [19]:
df_path_img_not_in_sample.shape

(7558, 1)

In [20]:
df_path_img_not_in_sample.head()

,path_img
0,../cell_images/Parasitized\C100P61ThinF_IMG_20...
1,../cell_images/Parasitized\C100P61ThinF_IMG_20...
2,../cell_images/Parasitized\C100P61ThinF_IMG_20...
3,../cell_images/Parasitized\C100P61ThinF_IMG_20...
4,../cell_images/Parasitized\C100P61ThinF_IMG_20...


In [21]:
df_path_img_not_in_sample.sample(1).values

array([['../cell_images/Parasitized\\C109P70ThinF_IMG_20150930_103811_cell_159.png']],
      dtype=object)